In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load Zillow's Home Value Index data by county
zillow_hvi = pd.read_csv('./Input_Data/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')  # Update with actual file path

# Inspect the first few rows of each dataset
display(zillow_hvi.head())

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30
0,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,208647.919509,...,865153.204373,864132.886760,859326.994934,855511.376681,856150.753342,859330.357288,861428.532750,865092.978391,870313.709819,876958.430862
1,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,147507.033889,...,295574.379458,295807.192380,296750.247771,298765.785128,301413.876693,303248.683214,304304.208302,304962.575753,305787.824375,306458.976441
2,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,109138.965088,...,279592.471752,279703.229912,280164.028963,281162.470003,282201.612312,282791.048604,282673.306736,282304.781165,281966.974553,281752.789691
3,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",4,13,143468.389801,...,465560.641583,465924.628213,466463.367772,467916.658718,469691.605770,471094.123500,471262.762845,470658.091259,469541.963287,468595.683794
4,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",6,73,214455.519985,...,902390.480071,904658.344283,909191.241655,917402.267397,927651.265767,936425.664757,940920.734698,942366.615764,941928.201959,941480.058616


In [3]:
# rename columns
zillow_hvi = zillow_hvi.rename(columns = {
    'RegionName' : 'county',
    'State' : 'state'
})
# standardize
zillow_hvi['county'] = zillow_hvi['county'].replace({'St.': 'Saint'}, regex=True)

# Pad the 'StateCodeFIPS' column with one zero (to ensure two-digit values)
zillow_hvi['StateCodeFIPS'] = zillow_hvi['StateCodeFIPS'].astype(str).str.zfill(2)

# Pad the 'MunicipalCodeFIPS' column with appropriate zeros (to ensure three-digit values)
zillow_hvi['MunicipalCodeFIPS'] = zillow_hvi['MunicipalCodeFIPS'].astype(str).str.zfill(3)

# Combine 'StateCodeFIPS' and 'MunicipalCodeFIPS' to create 'FIPS_Code'
zillow_hvi['FIPS_Code'] = zillow_hvi['StateCodeFIPS'] + zillow_hvi['MunicipalCodeFIPS']


# rename columns
zillow_hvi = zillow_hvi.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS'])
# Reset the index
zillow_hvi.reset_index(drop=True, inplace=True)

# filter dates
date_columns = [col for col in zillow_hvi.columns if '2017-01-01' <= col <= '2024-09-30']
# select only the 'count', 'state', and desired date columns
zillow_hvi = zillow_hvi[['county', 'state', 'FIPS_Code'] + date_columns]

display(zillow_hvi)

,county,state,FIPS_Code,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,...,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30
0,Los Angeles County,CA,06037,500739.067072,512265.720613,525432.954237,535235.001767,541905.251167,547183.809351,547091.200459,...,865153.204373,864132.886760,859326.994934,855511.376681,856150.753342,859330.357288,861428.532750,865092.978391,870313.709819,876958.430862
1,Cook County,IL,17031,214336.786235,216044.755353,217843.001214,219067.805391,219855.914494,219700.507242,219604.656354,...,295574.379458,295807.192380,296750.247771,298765.785128,301413.876693,303248.683214,304304.208302,304962.575753,305787.824375,306458.976441
2,Harris County,TX,48201,178716.520791,179142.734603,179713.829152,180454.198618,181142.208705,181747.020539,182121.484487,...,279592.471752,279703.229912,280164.028963,281162.470003,282201.612312,282791.048604,282673.306736,282304.781165,281966.974553,281752.789691
3,Maricopa County,AZ,04013,246123.477002,247443.887317,248893.537880,250429.463859,251869.098145,253286.488765,254652.572474,...,465560.641583,465924.628213,466463.367772,467916.658718,469691.605770,471094.123500,471262.762845,470658.091259,469541.963287,468595.683794
4,San Diego County,CA,06073,528137.644151,532410.296663,535354.317144,538452.810873,541037.626802,544379.174313,546783.561066,...,902390.480071,904658.344283,909191.241655,917402.267397,927651.265767,936425.664757,940920.734698,942366.615764,941928.201959,941480.058616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069,Banner County,NE,31007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,312795.550833,311485.589318,310145.852188,311434.732864,314227.660894,317507.187148,318169.453862,320202.514556,326574.757511,336171.914864
3070,Daggett County,UT,49009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,322240.756357,321559.613061,322228.733665,323090.997451,323151.629543,321829.784631,319723.202339,317503.341522,317439.546636,319855.715781
3071,Thomas County,NE,31171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,139958.059444,139785.504768,139589.980836,139762.032361,140190.358281,140476.529677,140761.685997,140592.818504,141053.159103,141689.697397
3072,McPherson County,NE,31117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,271622.786132,273287.520154,275442.831255,279388.685933,284655.115896,289031.217564,291360.915393,293819.568101,300835.519911,310842.045199


In [4]:
# Unpivot the DataFrame using pd.melt()
zillow_hvi_long = zillow_hvi.melt(
    id_vars=["county", "state", "FIPS_Code"],   # Columns to keep fixed
    var_name="date",                            # Name for the new 'date' column
    value_name="home_price"                     # Name for the new 'home_price' column
)

# Convert the 'date' column to a datetime format if needed
zillow_hvi_long['date'] = pd.to_datetime(zillow_hvi_long['date'])

In [5]:
zillow_hvi_long

,county,state,FIPS_Code,date,home_price
0,Los Angeles County,CA,06037,2017-01-31,500739.067072
1,Cook County,IL,17031,2017-01-31,214336.786235
2,Harris County,TX,48201,2017-01-31,178716.520791
3,Maricopa County,AZ,04013,2017-01-31,246123.477002
4,San Diego County,CA,06073,2017-01-31,528137.644151
...,...,...,...,...,...
285877,Banner County,NE,31007,2024-09-30,336171.914864
285878,Daggett County,UT,49009,2024-09-30,319855.715781
285879,Thomas County,NE,31171,2024-09-30,141689.697397
285880,McPherson County,NE,31117,2024-09-30,310842.045199


In [6]:
# export cleaned data
zillow_hvi_long.to_csv('./Tableau/Datasources/clean_home_price.csv', index=False)